# IR PACKAGE - Music Recommendation System

We propose a novel approach to music recommendation that takes the audio of the user as input, converts it to text and performs sentiment analysis, tf-idf, and normalization. We then use content-based and collaborative filtering techniques to recommend songs that are likely to be enjoyed by the user based on their expressed sentiments and preferences. By leveraging user-generated content, our approach offers a more personalized and relevant music recommendation experience

# Pre requisites

In [ ]:
import json
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.6/238.6 KB 13.0 MB/s eta 0:00:00


In [ ]:
# Credentials 

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id='23d70a1ad9154551b8e3e52577eeebe9',
    client_secret='60f1dcf960fb4c0598eb2354253bd93d'))

In [ ]:
def playlist_analyzer(user, playlist_id):    
    names = []
    uris = []
    artists = []
    popularity = []
    danceability = []
    energy = []
    key = []
    loudness = []
    mode = []
    speechiness = []
    acousticness = []
    instrumentalness =[]
    liveness=[]
    valence = []
    tempo =[]
    duration_ms =[]
    time_signature =[]
    playlist = spotify.user_playlist_tracks(user=user, playlist_id=playlist_id)
    tracks = playlist['items']
    
    while playlist['next']: # While statement to switch to next page of the playlist
        playlist = spotify.next(playlist)
        tracks.extend(playlist['items'])
   
    for item in tracks:
        names.append(item["track"]["name"])
        uri = item["track"]["uri"]
        uris.append(uri)
        popularity.append(item["track"]["popularity"])
        artists.append([artist["name"] for artist in item["track"]["artists"]])
        features = spotify.audio_features(uri)[0]
        danceability.append(features["danceability"])
        energy.append(features["energy"])
        key.append(features["key"])
        loudness.append(features["loudness"])
        mode.append(features["mode"])
        speechiness.append(features["speechiness"])
        acousticness.append(features["acousticness"])
        instrumentalness.append(features["instrumentalness"])
        liveness.append(features["liveness"])
        valence.append(features["valence"])
        tempo.append(features["tempo"])
        duration_ms.append(features["duration_ms"])
        time_signature.append(features["time_signature"])
    
    # Creating a df from with the song information
    df = pd.DataFrame({"names":names, 
                      "uri": uris,
                      "artists": artists,
                      "popularity": popularity,
                        "danceability":danceability,
                       "energy":energy,
                       "key": key,
                       "loudness":loudness,
                       "mode": mode,
                       "speechiness":speechiness,
                       "acousticness":acousticness,
                       "instrumentalness":instrumentalness,
                       "liveness": liveness,
                       "valence":valence,
                       "tempo": tempo,
                       "duration_ms":duration_ms,
                       "time_signature": time_signature})

    return df

# Audio query to text query

In [ ]:
!sudo apt-get install libportaudio2
!pip install sounddevice

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 65.4 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Fetched 65.4 kB in 0s (194 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 128285 

In [ ]:
!pip install sounddevice soundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import Libraries
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import nltk
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio, sr = get_audio()

In [ ]:
import scipy
scipy.io.wavfile.write('recording.wav', sr, audio)

In [ ]:
!pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [ ]:
import speech_recognition as sr

# Set the path to your audio file
audio_file_path = '/content/recording.wav'

# Create a recognizer object
r = sr.Recognizer()

# Load the audio file
with sr.AudioFile(audio_file_path) as source:
    audio = r.record(source)

# Transcribe the audio file
text = r.recognize_google(audio)

# Print the transcribed text
print(text)


In [ ]:
# Tokenize the input text
tokens = nltk.word_tokenize(text)

# Get the set of English stopwords
stop_words = set(stopwords.words('english'))

# Remove the stopwords from the token list
filtered_tokens = [token for token in tokens if not token.lower() in stop_words]

# Join the filtered tokens back into a string
filtered_text = ' '.join(filtered_tokens)

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
lemmatizer = WordNetLemmatizer()  
text = [lemmatizer.lemmatize(word) for word in filtered_text]   
#joining the sentences
cleanedLine=" "
cleanedLine=cleanedLine.join(text)

In [ ]:
print(cleanedLine)

a d o r e   a d o r e


# Search engine

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
https://open.spotify.com/playlist/4ulHIyJD1fn3HnmFaD0ZMC

In [ ]:
#General playlist -> feature extraction
df = playlist_analyzer("spotify", "https://open.spotify.com/playlist/0X9hkrRqCCP69Ze1MheAda?si=099e5aeff89140b7")

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   names             600 non-null    object 
 1   uri               600 non-null    object 
 2   artists           600 non-null    object 
 3   popularity        600 non-null    int64  
 4   danceability      600 non-null    float64
 5   energy            600 non-null    float64
 6   key               600 non-null    int64  
 7   loudness          600 non-null    float64
 8   mode              600 non-null    int64  
 9   speechiness       600 non-null    float64
 10  acousticness      600 non-null    float64
 11  instrumentalness  600 non-null    float64
 12  liveness          600 non-null    float64
 13  valence           600 non-null    float64
 14  tempo             600 non-null    float64
 15  duration_ms       600 non-null    int64  
 16  time_signature    600 non-null    int64  
dt

In [ ]:
df.to_csv("general.csv", index=False)

In [ ]:
vectorizer = TfidfVectorizer()
test=pd.concat([df['names'],df['artists']], join = 'outer', axis = 1)
X = vectorizer.fit_transform(test)

In [ ]:
li=[]
for i in range(0,600):
    li.append(df.loc[i, 'names']+" "+df.loc[i,'artists'][0])

df['names+artist']=li
X = vectorizer.fit_transform(df['names+artist'])

In [ ]:
query = "Calm Down (with Selena Gomez)"
query_vec = vectorizer.transform([query])

In [ ]:
results = cosine_similarity(X,query_vec)

In [ ]:
results

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [ ]:
df['cosine']=np.array(results)
sort_df = df.sort_values(by = 'cosine', ascending=False)
sort_df.head(10)

,names,uri,artists,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,names+artist,cosine
127,Down,spotify:track:7LP4Es66zdY7CyjepqmvAg,"[Jay Sean, Lil Wayne]",1,0.657,0.695,2,-4.493,1,0.0321,0.01080,0.000000,0.0822,0.683,65.997,212107,4,Down Jay Sean,0.398676
286,Kryptonite,spotify:track:6ZOBP3NvffbU4SZcrnt1k6,[3 Doors Down],78,0.545,0.865,11,-5.708,0,0.0286,0.00664,0.000011,0.1680,0.543,99.009,233933,4,Kryptonite 3 Doors Down,0.372645
332,Don't Let Me Down,spotify:track:0w75KG0DuLDi6tdHmEMovD,"[The Chainsmokers, Daya]",48,0.534,0.868,11,-5.057,1,0.2050,0.18700,0.003200,0.1480,0.406,159.763,208667,4,Don't Let Me Down The Chainsmokers,0.333493
79,Upside Down,spotify:track:3pbtBomO4Zt5gGiqsYeiBH,[Diana Ross],67,0.873,0.855,5,-8.559,0,0.0615,0.17900,0.028500,0.0377,0.884,107.868,245600,4,Upside Down Diana Ross,0.328301
520,Here Without You,spotify:track:3NLrRZoMF0Lx6zTlYqeIo4,[3 Doors Down],74,0.557,0.533,10,-6.817,0,0.0252,0.04920,0.000000,0.2050,0.233,143.994,238733,4,Here Without You 3 Doors Down,0.327593
435,Shake You Down,spotify:track:7wQ9alB79WZb0F5gFLbxSh,[Gregory Abbott],56,0.787,0.474,11,-10.384,0,0.0265,0.52300,0.000000,0.2350,0.826,94.995,245413,4,Shake You Down Gregory Abbott,0.303476
218,Down Under,spotify:track:46RVKt5Edm1zl0rXhPJZxz,[Men At Work],69,0.746,0.790,11,-4.700,0,0.0257,0.04850,0.026100,0.0534,0.891,107.231,220867,4,Down Under Men At Work,0.289312
294,Rock With You,spotify:track:0oA7AdpU9Uhg1GL7p739H6,[Michael Jackson],0,0.794,0.741,1,-6.494,1,0.0394,0.07930,0.000292,0.1160,0.856,114.489,220040,4,Rock With You Michael Jackson,0.275515
563,Lay Down Sally,spotify:track:6io0LJIP9ntX35hTfaxwQG,[Eric Clapton],60,0.690,0.500,4,-16.145,0,0.0909,0.59800,0.028800,0.1100,0.586,191.539,236093,4,Lay Down Sally Eric Clapton,0.268785
285,Can't Nobody Hold Me Down (feat. Mase),spotify:track:1EM23A621Z0QyTlWKu7T1l,"[Diddy, Mase]",57,0.928,0.771,3,-5.935,0,0.2510,0.10200,0.000011,0.3380,0.679,93.987,231160,4,Can't Nobody Hold Me Down (feat. Mase) Diddy,0.255366


# Content based

In [ ]:
#Playlist 
playlist_df = playlist_analyzer("spotify", "https://open.spotify.com/playlist/4hMcqod7ERKJ9mtjgdimeV?si=00bfd8719a074aa7")
features_playlist = df.iloc[:,4:17].copy() #Creating a subset of the df with only the features 

In [ ]:
playlist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   names             225 non-null    object 
 1   uri               225 non-null    object 
 2   artists           225 non-null    object 
 3   popularity        225 non-null    int64  
 4   danceability      225 non-null    float64
 5   energy            225 non-null    float64
 6   key               225 non-null    int64  
 7   loudness          225 non-null    float64
 8   mode              225 non-null    int64  
 9   speechiness       225 non-null    float64
 10  acousticness      225 non-null    float64
 11  instrumentalness  225 non-null    float64
 12  liveness          225 non-null    float64
 13  valence           225 non-null    float64
 14  tempo             225 non-null    float64
 15  duration_ms       225 non-null    int64  
 16  time_signature    225 non-null    int64  
dt

In [ ]:
playlist_df.head(20)

,names,uri,artists,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Boy's a liar Pt. 2,spotify:track:6AQbmUe0Qwf5PZnt4HmTXv,"[PinkPantheress, Ice Spice]",95,0.696,0.809,5,-8.254,1,0.0500,0.25200,0.000128,0.2480,0.8570,132.962,131013,4
1,Die For You - Remix,spotify:track:7oDd86yk8itslrA9HRP2ki,"[The Weeknd, Ariana Grande]",94,0.531,0.525,1,-6.500,0,0.0671,0.23200,0.000000,0.4410,0.5020,66.900,232857,4
2,Flowers,spotify:track:0yLdNVWF3Srea0uzk55zFn,[Miley Cyrus],100,0.707,0.681,0,-4.325,1,0.0668,0.06320,0.000005,0.0322,0.6460,117.999,200455,4
3,Unholy (feat. Kim Petras),spotify:track:3nqQXoyQOWXiESFLlDF1hG,"[Sam Smith, Kim Petras]",92,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.2380,131.121,156943,4
4,Kill Bill,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,[SZA],93,0.644,0.735,8,-5.747,1,0.0391,0.05210,0.144000,0.1610,0.4180,88.980,153947,4
5,I'm Good (Blue),spotify:track:4uUG5RXrOk84mYEfFvj3cK,"[David Guetta, Bebe Rexha]",93,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.3040,128.040,175238,4
6,Calm Down (with Selena Gomez),spotify:track:0WtM2NBVQNNJLh6scP13H8,"[Rema, Selena Gomez]",93,0.801,0.806,11,-5.206,1,0.0381,0.38200,0.000669,0.1140,0.8020,106.999,239318,4
7,Creepin' (with The Weeknd & 21 Savage),spotify:track:2dHHgzDwk4BJdRwy9uXhTO,"[Metro Boomin, The Weeknd, 21 Savage]",94,0.715,0.620,1,-6.005,0,0.0484,0.41700,0.000000,0.0822,0.1720,97.950,221520,4
8,Escapism.,spotify:track:5Z2MiIZ5I3jJvvmeWMLbOQ,"[RAYE, 070 Shake]",90,0.538,0.742,2,-5.355,1,0.1140,0.13800,0.000047,0.0934,0.2500,96.107,272373,4
9,Here With Me,spotify:track:78Sw5GDo6AlGwTwanjXbGh,[d4vd],92,0.574,0.469,4,-8.209,1,0.0254,0.53400,0.000092,0.1280,0.2880,132.023,242485,4


In [ ]:
print(features_playlist)

     danceability  energy  key  loudness  mode  speechiness  acousticness  \
0           0.533   0.638    4    -7.130     1       0.0341       0.20200   
1           0.609   0.923    9    -3.908     1       0.0338       0.16000   
2           0.549   0.529    3   -12.291     0       0.1080       0.76000   
3           0.856   0.609    0    -7.223     1       0.0824       0.00801   
4           0.577   0.462    2    -7.989     1       0.0279       0.12800   
..            ...     ...  ...       ...   ...          ...           ...   
595         0.376   0.246   10    -9.803     1       0.0286       0.86200   
596         0.724   0.907   11    -4.780     0       0.0632       0.01870   
597         0.631   0.932    5    -4.142     1       0.0354       0.04360   
598         0.528   0.863    6    -5.772     0       0.0368       0.06160   
599         0.573   0.902    6    -7.638     0       0.0540       0.01800   

     instrumentalness  liveness  valence    tempo  duration_ms  time_signat

In [ ]:
#General playlist creation
playlist_df.to_csv("collab.csv", index=False)

In [ ]:
# Current top songs
Hothits_df = playlist_analyzer("spotify", "https://open.spotify.com/playlist/37i9dQZF1DX0ieekvzt1Ic?si=dfd6d0e9ec974915")
Hothits_features_playlist = playlist_df.iloc[:,4:17].copy() #Creating a subset of the df with only the features 

In [ ]:
Hothits_df.to_csv("hothits.csv", index=False) # hothits playlist

In [ ]:
print(Hothits_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   names             51 non-null     object 
 1   uri               51 non-null     object 
 2   artists           51 non-null     object 
 3   popularity        51 non-null     int64  
 4   danceability      51 non-null     float64
 5   energy            51 non-null     float64
 6   key               51 non-null     int64  
 7   loudness          51 non-null     float64
 8   mode              51 non-null     int64  
 9   speechiness       51 non-null     float64
 10  acousticness      51 non-null     float64
 11  instrumentalness  51 non-null     float64
 12  liveness          51 non-null     float64
 13  valence           51 non-null     float64
 14  tempo             51 non-null     float64
 15  duration_ms       51 non-null     int64  
 16  time_signature    51 non-null     int64  
dtyp

In [ ]:
li=[]
for i in range(0,51):
    li.append(Hothits_df.loc[i, 'names']+" "+Hothits_df.loc[i,'artists'][0])

Hothits_df['names+artist']=li

In [ ]:
combined = pd.concat([df,Hothits_df])

In [ ]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 651 entries, 0 to 50
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   names             651 non-null    object 
 1   uri               651 non-null    object 
 2   artists           651 non-null    object 
 3   popularity        651 non-null    int64  
 4   danceability      651 non-null    float64
 5   energy            651 non-null    float64
 6   key               651 non-null    int64  
 7   loudness          651 non-null    float64
 8   mode              651 non-null    int64  
 9   speechiness       651 non-null    float64
 10  acousticness      651 non-null    float64
 11  instrumentalness  651 non-null    float64
 12  liveness          651 non-null    float64
 13  valence           651 non-null    float64
 14  tempo             651 non-null    float64
 15  duration_ms       651 non-null    int64  
 16  time_signature    651 non-null    int64  
 17

In [ ]:
features_playlist = combined.iloc[:,4:17].copy() #Creating a subset of the df with only the features 

In [ ]:
# Initialize the model, fit and transform data
scaler = StandardScaler()
scaler.fit(features_playlist)
features_scaled = scaler.transform(features_playlist)
features_scaled_df = pd.DataFrame(features_playlist)

In [ ]:
features_scaled_df.head(3)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.533,0.638,4,-7.130,1,0.0341,0.202,0.000000,0.0729,0.937,156.663,153760,4
1,0.609,0.923,9,-3.908,1,0.0338,0.160,0.000005,0.2950,0.961,115.996,294987,4
2,0.549,0.529,3,-12.291,0,0.1080,0.760,0.000000,0.2060,0.464,82.755,184333,4


In [ ]:
features_scaled_df[['loudness','key']].shape

(651, 2)

In [ ]:
# Initializing the model, fit and predict clusters
kmeans = KMeans()
kmeans.fit(features_scaled_df[['loudness','key']])
clusters = kmeans.predict(features_scaled_df[['loudness','key']])
combined["cluster"]=clusters

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# Check the size of the clusters
pd.Series(clusters).value_counts().sort_index()

0     66
1     89
2     94
3     55
4     38
5     97
6    113
7     99
dtype: int64

In [ ]:
combined.head(10)

,names,uri,artists,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,names+artist,cluster
0,The Twist,spotify:track:3ohLnESFgYACPMCkoTOzqE,[Chubby Checker],51,0.533,0.638,4,-7.130,1,0.0341,0.20200,0.000000,0.0729,0.937,156.663,153760,4,The Twist Chubby Checker,2
1,Smooth (feat. Rob Thomas),spotify:track:0n2SEXB2qoRQg171q7XqeW,"[Santana, Rob Thomas]",67,0.609,0.923,9,-3.908,1,0.0338,0.16000,0.000005,0.2950,0.961,115.996,294987,4,Smooth (feat. Rob Thomas) Santana,5
2,Mack the Knife,spotify:track:3E5ndyOfO6vFDEIE42HA8o,[Bobby Darin],60,0.549,0.529,3,-12.291,0,0.1080,0.76000,0.000000,0.2060,0.464,82.755,184333,4,Mack the Knife Bobby Darin,3
3,Uptown Funk (feat. Bruno Mars),spotify:track:32OlwWuMpZ6b0aN2RZOeMS,"[Mark Ronson, Bruno Mars]",80,0.856,0.609,0,-7.223,1,0.0824,0.00801,0.000082,0.0344,0.928,114.988,269667,4,Uptown Funk (feat. Bruno Mars) Mark Ronson,6
4,How Do I Live,spotify:track:7BD50ATrF3Vab5FQy7vtK8,[LeAnn Rimes],67,0.577,0.462,2,-7.989,1,0.0279,0.12800,0.000000,0.0822,0.258,128.303,266973,4,How Do I Live LeAnn Rimes,1
5,Party Rock Anthem,spotify:track:0IkKz2J93C94Ei4BvDop7P,"[LMFAO, Lauren Bennett, GoonRock]",67,0.750,0.727,5,-4.210,0,0.1420,0.01890,0.000000,0.2660,0.359,129.993,262173,4,Party Rock Anthem LMFAO,2
6,I Gotta Feeling,spotify:track:4vp2J1l5RD4gMZwGFLfRAu,[Black Eyed Peas],5,0.743,0.766,0,-6.375,1,0.0265,0.08730,0.000000,0.5090,0.610,127.960,289133,4,I Gotta Feeling Black Eyed Peas,6
7,Macarena - Bayside Boys Remix,spotify:track:2df5QsXucx4VLiHNGusXD5,[Los Del Rio],55,0.737,0.930,1,-6.083,1,0.0534,0.23700,0.000002,0.0911,0.965,103.191,253107,4,Macarena - Bayside Boys Remix Los Del Rio,6
8,Shape of You,spotify:track:7qiZfU4dY1lWllzX7mPBI3,[Ed Sheeran],85,0.825,0.652,1,-3.183,0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,233713,4,Shape of You Ed Sheeran,6
9,Physical,spotify:track:3hVrI7snZMTmHCnbwME47u,[Olivia Newton-John],0,0.834,0.633,4,-11.442,0,0.0422,0.06310,0.000050,0.3130,0.904,123.815,224773,4,Physical Olivia Newton-John,3


In [ ]:
df=df.drop(['cosine'],axis=1)

In [ ]:
playlist_df.to_csv("clustered.csv", index=False) # clustered playlist
#features_scaled_df.to_csv("features_scales_df.csv", index=False)

In [ ]:
https://open.spotify.com/playlist/37i9dQZF1DX0ieekvzt1Ic?si=dfd6d0e9ec974915

In [ ]:
u_labels = np.unique(df['cluster'])
print(u_labels)

[0 1]


In [ ]:
droped=df.drop(['names','uri','artists'],axis=1)

# Collaborative 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer()

In [ ]:
import librosa
import numpy as np
from scipy import stats
import pandas as pd
import glob
import spotipy
import spotipy.util as util
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from pathlib import Path
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import pickle
import math
import numpy as np
from nltk.tree import *
from nltk.stem import WordNetLemmatizer 
import random
import numpy as num
from math import exp
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer 

In [ ]:
token="BQBFFqEfiu4G7O2Y37NtYLu07mXmk_iCLo4KLdw87wshGCLPx8F1Gd1SpBGejLs9yL3NcJh21ZTuRqFvsUHNBfOPhzWss6r44pWvRYBqKchBfUgN_1vHu0FCkFYY6klc3rxABttJiExhUdC2A0fFFnmMJc4daxRHEnwDRNcKSJiLH5pkN3Em0LGF6pXtsUZjFpA9745fYA"

In [ ]:
sp = spotipy.Spotify(auth="BQBhz6fIu8gzIrUHMph37l9u9EEQ47nCQvmwWuwrsb4DGEhUjyWp46AgwpB0qiCRTs7eNKY65gVsvXAIt4tvfM2_33u4QlgMimDchv8qRdeNednVGFTvRZ9ncKQHRDqZY-vLnPWxCM800l3T04fh_qwueyEKPRatmFyhWrytVih7qXFaSkMQjHDHIplanlXlKCPmOq194g")

In [ ]:
results = sp.current_user_top_tracks(limit=1000, offset=0,time_range='short_term')

In [ ]:
# Convert it to Dataframe
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_favourite = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_favourite.head(20)

,track_name,album,track_id,artist,duration,popularity
0,willow,evermore,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,214706,75
1,Idhayathai Yedho Ondru,Yennai Arindhaal (Original Motion Picture Soun...,23bpCCVsTUKviTEfoy4tbM,Harris Jayaraj,235391,54
2,The Hills,Beauty Behind The Madness,7fBv7CLKzipRk6EC6TWHOB,The Weeknd,242253,85
3,Vaa Vaathi (Dhanush Reprise Version),Vaathi,43bgjI3ZddPJsu3JvNmYui,G. V. Prakash,223934,63
4,Koodamela Koodavechi,Rummy (Original Motion Picture Soundtrack),102YVYHLNfgkfQP3qvDqsD,D. Imman,300373,55
5,Azhakooril,Thirumalai (Original Motion Picture Soundtrack),7BsQNB20nHHZY9pNcrrrsI,S. P. Balasubrahmanyam,272826,53
6,Nenjukkule,Kadal (Original Motion Picture Soundtrack),7FEWG1bL31YAcnxJQvxJJO,A.R. Rahman,290000,55
7,Kannan Varum Velai,Deepavali (Original Motion Picture Soundtrack),72niNpc1LP2DEqaF118U7M,Anuradha Sriram,228066,42
8,London Boy,Lover,1LLXZFeAHK9R4xUramtUKw,Taylor Swift,190240,72
9,Nallai Allai,Kaatru Veliyidai (Original Motion Picture Soun...,7gwuIhFmNaf3wcRWG1WadD,A.R. Rahman,239507,55


In [ ]:
# Creating a function get features of each track from track id
def getTrackFeatures(track_id):
  meta = sp.track(track_id)
  features = sp.audio_features(track_id)

  # meta
  track_id = track_id
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [track_id, name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  print(track)
  return track

In [ ]:
# Getting track features for each song in favorite song dataframe
fav_tracks = []
for track in df_favourite['track_id']:
    try:  
        track = getTrackFeatures(track)
        fav_tracks.append(track)
    except:
        pass

['0lx2cLdOt3piJbcaXIV74f', 'willow', 'evermore', 'Taylor Swift', '2020-12-11', 214706, 75, 0.392, 0.833, 0.574, 0.00179, 0.145, -9.195, 0.17, 81.112, 4]
['23bpCCVsTUKviTEfoy4tbM', 'Idhayathai Yedho Ondru', 'Yennai Arindhaal (Original Motion Picture Soundtrack)', 'Harris Jayaraj', '2015-01-04', 235391, 54, 0.721, 0.845, 0.496, 0, 0.11, -8.063, 0.0508, 131.969, 3]
['7fBv7CLKzipRk6EC6TWHOB', 'The Hills', 'Beauty Behind The Madness', 'The Weeknd', '2015-08-28', 242253, 85, 0.585, 0.0671, 0.564, 0, 0.135, -7.063, 0.0515, 113.003, 4]
['43bgjI3ZddPJsu3JvNmYui', 'Vaa Vaathi (Dhanush Reprise Version)', 'Vaathi', 'G. V. Prakash', '2023-02-23', 223934, 63, 0.59, 0.605, 0.368, 0, 0.671, -12.03, 0.0306, 91.05, 4]
['102YVYHLNfgkfQP3qvDqsD', 'Koodamela Koodavechi', 'Rummy (Original Motion Picture Soundtrack)', 'D. Imman', '2013-10-25', 300373, 55, 0.694, 0.324, 0.695, 0, 0.177, -5.035, 0.0276, 120.057, 4]
['7BsQNB20nHHZY9pNcrrrsI', 'Azhakooril', 'Thirumalai (Original Motion Picture Soundtrack)', 'Vid

In [ ]:
# Create favorite track wtih audio features dataset
df_fav = pd.DataFrame(fav_tracks, columns = ['track_id', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df_fav.head(10)

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,0lx2cLdOt3piJbcaXIV74f,willow,evermore,Taylor Swift,2020-12-11,214706,75,0.392,0.8330,0.574,0.001790,0.145,-9.195,0.1700,81.112,4
1,23bpCCVsTUKviTEfoy4tbM,Idhayathai Yedho Ondru,Yennai Arindhaal (Original Motion Picture Soun...,Harris Jayaraj,2015-01-04,235391,54,0.721,0.8450,0.496,0.000000,0.110,-8.063,0.0508,131.969,3
2,7fBv7CLKzipRk6EC6TWHOB,The Hills,Beauty Behind The Madness,The Weeknd,2015-08-28,242253,85,0.585,0.0671,0.564,0.000000,0.135,-7.063,0.0515,113.003,4
3,43bgjI3ZddPJsu3JvNmYui,Vaa Vaathi (Dhanush Reprise Version),Vaathi,G. V. Prakash,2023-02-23,223934,63,0.590,0.6050,0.368,0.000000,0.671,-12.030,0.0306,91.050,4
4,102YVYHLNfgkfQP3qvDqsD,Koodamela Koodavechi,Rummy (Original Motion Picture Soundtrack),D. Imman,2013-10-25,300373,55,0.694,0.3240,0.695,0.000000,0.177,-5.035,0.0276,120.057,4
5,7BsQNB20nHHZY9pNcrrrsI,Azhakooril,Thirumalai (Original Motion Picture Soundtrack),Vidyasagar,2003-04-13,272826,53,0.473,0.0905,0.659,0.000040,0.168,-9.292,0.0477,94.817,4
6,7FEWG1bL31YAcnxJQvxJJO,Nenjukkule,Kadal (Original Motion Picture Soundtrack),A.R. Rahman,2012-12-14,290000,55,0.473,0.6990,0.400,0.000013,0.132,-8.224,0.0311,154.040,3
7,72niNpc1LP2DEqaF118U7M,Kannan Varum Velai,Deepavali (Original Motion Picture Soundtrack),Yuvan Shankar Raja,2007-01-05,228066,42,0.795,0.4970,0.508,0.014700,0.645,-9.838,0.0594,150.365,3
8,1LLXZFeAHK9R4xUramtUKw,London Boy,Lover,Taylor Swift,2019-08-23,190240,72,0.695,0.0246,0.710,0.000104,0.133,-6.639,0.0500,157.925,4
9,7gwuIhFmNaf3wcRWG1WadD,Nallai Allai,Kaatru Veliyidai (Original Motion Picture Soun...,A.R. Rahman,2017-03-20,239507,55,0.579,0.8030,0.355,0.000000,0.043,-9.323,0.0253,101.040,4


In [ ]:
df_fav.to_csv('Contentbased.csv')

In [ ]:
df_fav = df_fav.drop(columns=['name', 'album', 'artist', 'release_date'])
df_fav.head()

,track_id,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,0lx2cLdOt3piJbcaXIV74f,214706,75,0.392,0.8330,0.574,0.00179,0.145,-9.195,0.1700,81.112,4
1,23bpCCVsTUKviTEfoy4tbM,235391,54,0.721,0.8450,0.496,0.00000,0.110,-8.063,0.0508,131.969,3
2,7fBv7CLKzipRk6EC6TWHOB,242253,85,0.585,0.0671,0.564,0.00000,0.135,-7.063,0.0515,113.003,4
3,43bgjI3ZddPJsu3JvNmYui,223934,63,0.590,0.6050,0.368,0.00000,0.671,-12.030,0.0306,91.050,4
4,102YVYHLNfgkfQP3qvDqsD,300373,55,0.694,0.3240,0.695,0.00000,0.177,-5.035,0.0276,120.057,4


In [ ]:
df_fav['favorite'] = 1

In [ ]:
li=[]
for i in range(0,50):
    li.append(df_fav.loc[i, 'name']+" "+df_fav.loc[i,'artist'])

print(li)
df_fav['name+artist']=li
X = vectorizer.fit_transform(df_fav['name+artist'])

['willow Taylor Swift', 'Idhayathai Yedho Ondru Harris Jayaraj', 'The Hills The Weeknd', 'Vaa Vaathi (Dhanush Reprise Version) G. V. Prakash', 'Koodamela Koodavechi D. Imman', 'Azhakooril Vidyasagar', 'Nenjukkule A.R. Rahman', 'Kannan Varum Velai Yuvan Shankar Raja', 'London Boy Taylor Swift', 'Nallai Allai A.R. Rahman', 'Sara Sara Saara Kathu Ghibran', 'Pogatha Yennavittu Sam C.S.', 'Agayam Theepiditha - From "Madras" Pradeep Kumar', 'Mogathirai - From "Pizza" Pradeep Kumar', 'Ambikapathy A.R. Rahman', 'Blinding Lights The Weeknd', 'The Nights Avicii', 'Nanbane Yuvan Shankar Raja', 'Photograph Ed Sheeran', 'Ninaivirukka (From "Pathu Thala") A.R. Rahman', 'Dimple BTS', 'Kho Gaye Mismatched', 'Sahana Saral Thoo A.R. Rahman', 'Unnai Kaanadhu Naan Shankar-Ehsaan-Loy', 'Tera Chehra Himesh Reshammiya', 'Needhane Yuvan Shankar Raja', 'Kaathodu Kaathanen (From "Jail") G. V. Prakash', 'Kutti Puli Kootam Harris Jayaraj', 'Kombula Poova Ilaiyaraaja', 'Avalukena Anirudh Ravichander', 'Kho Gaye Hu

In [ ]:
query = "Pradeep Kumar"
query_vec = vectorizer.transform([query])

In [ ]:
results = cosine_similarity(X,query_vec)

In [ ]:
results

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.55816904],
       [0.6203695 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [ ]:
df_fav['cosine']=np.array(results)

In [ ]:
df_fav=df_fav.drop(['names+artist'], axis=1)

In [ ]:
sort_df = df_fav.sort_values(by = ['cosine', 'popularity'], ascending=False)
sort_df.head(10)

,track_id,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,cosine,name+artist
13,0JZPgDCigVlkSbKz16ec4l,"Mogathirai - From ""Pizza""",I Love Pradeep Kumar,Pradeep Kumar,2018-06-12,227017,47,0.522,0.63300,0.264,0.000040,0.1000,-9.428,0.0415,199.686,4,0.620369,"Mogathirai - From ""Pizza"" Pradeep Kumar"
12,0Ei1ZN3Fj6lkp9QV7Q20MR,"Agayam Theepiditha - From ""Madras""",I Love Pradeep Kumar,Pradeep Kumar,2018-06-12,230062,51,0.517,0.86900,0.423,0.000000,0.0785,-8.388,0.0389,117.867,4,0.558169,"Agayam Theepiditha - From ""Madras"" Pradeep Kumar"
15,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,After Hours,The Weeknd,2020-03-20,200040,89,0.514,0.00146,0.730,0.000095,0.0897,-5.934,0.0598,171.005,4,0.000000,Blinding Lights The Weeknd
46,09mEdoA6zrmBPgTEN5qXmN,Call Out My Name,"My Dear Melancholy,",The Weeknd,2018-03-30,228373,86,0.461,0.17000,0.593,0.000000,0.3070,-4.954,0.0356,134.170,3,0.000000,Call Out My Name The Weeknd
2,7fBv7CLKzipRk6EC6TWHOB,The Hills,Beauty Behind The Madness,The Weeknd,2015-08-28,242253,85,0.585,0.06710,0.564,0.000000,0.1350,-7.063,0.0515,113.003,4,0.000000,The Hills The Weeknd
16,0ct6r3EGTcMLPtrXHDvVjc,The Nights,The Days / Nights,Avicii,2014-01-01,176658,84,0.527,0.01660,0.835,0.000000,0.2490,-5.298,0.0433,125.983,4,0.000000,The Nights Avicii
18,1HNkqx9Ahdgi1Ixy2xkKkL,Photograph,x (Deluxe Edition),Ed Sheeran,2014-06-21,258986,83,0.614,0.60700,0.379,0.000464,0.0986,-10.480,0.0476,107.989,4,0.000000,Photograph Ed Sheeran
48,22VdIZQfgXJea34mQxlt81,Can't Feel My Face,Beauty Behind The Madness,The Weeknd,2015-08-28,213520,81,0.705,0.11300,0.769,0.000000,0.1050,-5.526,0.0425,107.949,4,0.000000,Can't Feel My Face The Weeknd
49,2P4OICZRVAQcYAV2JReRfj,Waiting For Love,Stories,Avicii,2015-10-02,230613,80,0.579,0.31000,0.736,0.000000,0.1980,-3.863,0.0527,127.999,4,0.000000,Waiting For Love Avicii
47,1SymEzIT3H8UZfibCs3TYi,Afterglow,Lover,Taylor Swift,2019-08-23,223293,76,0.756,0.13000,0.449,0.000000,0.1140,-8.746,0.0344,111.011,4,0.000000,Afterglow Taylor Swift
